In [1]:
!pip install networkx gensim

In [2]:
import networkx as nx
import numpy as np
import os
import time
import pandas as pd
from itertools import chain
from collections import defaultdict
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [3]:
def load_data(file_name):
    """
    read edges from an edge file
    """
    edges = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        user_id, friends = row["user_id"], eval(row["friends"])
        for friend in friends:
            # add each friend relation as an edge
            edges.append((user_id, friend))
    edges = sorted(edges)#得到list
    
    return edges

def generate_false_edges(true_edges, num_false_edges=5):
    """
    generate false edges given true edges
    """
    nodes = list(set(chain.from_iterable(true_edges)))
    true_edges = set(true_edges)
    false_edges = set()
    
    while len(false_edges) < num_false_edges:
        # randomly sample two different nodes and check whether the pair exisit or not
        head, tail = np.random.choice(nodes, 2)
        if head != tail and (head, tail) not in true_edges and (head, tail) not in false_edges:
            false_edges.add((head, tail))    
    false_edges = sorted(false_edges)
    
    return false_edges

def construct_graph_from_edges(edges):
    """
    generate a directed graph object given true edges
    DiGraph documentation: https://networkx.github.io/documentation/stable/reference/classes/digraph.html
    """
    # convert a list of edges {(u, v)} to a list of edges with weights {(u, v, w)}
    edge_weight = defaultdict(float)
    for e in edges:
        edge_weight[e] += 1.0
    weighed_edge_list = list()
    for e in sorted(edge_weight.keys()):
        weighed_edge_list.append((e[0], e[1], edge_weight[e]))
        
    graph = nx.DiGraph()
    graph.add_weighted_edges_from(weighed_edge_list)
    
    print("number of nodes:", graph.number_of_nodes())
    print("number of edges:", graph.number_of_edges())
    
    return graph

In [4]:
def alias_setup(probs): 
    """
    compute utility lists for non-uniform sampling from discrete distributions.
    details: https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    """
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = list()
    larger = list()
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def get_alias_node(graph, node):
    """
    get the alias node setup lists for a given node.
    """
    # get the unnormalized probabilities with the first-order information
    unnormalized_probs = list()
    for nbr in graph.neighbors(node):
        unnormalized_probs.append(graph[node][nbr]['weight'])#若不是weighted，填1即可
    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs
        
    return alias_setup(normalized_probs)
    
def get_alias_edge(graph, src, dst, p=1, q=1):
    """
    get the alias edge setup lists for a given edge.
    """
    # get the unnormalized probabilities with the second-order information
    unnormalized_probs = list()
    for dst_nbr in graph.neighbors(dst):
        if dst_nbr == src: # distance is 0
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'] / p)
        elif graph.has_edge(dst_nbr, src): # distance is 1
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'])
        else: # distance is 2
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'] / q)
    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs

    return alias_setup(normalized_probs)

def preprocess_transition_probs(graph, p=1, q=1):
    """
    preprocess transition probabilities for guiding the random walks.
    """
    alias_nodes = dict()
    for node in graph.nodes():
        alias_nodes[node] = get_alias_node(graph, node)

    alias_edges = dict()
    for edge in graph.edges():
        alias_edges[edge] = get_alias_edge(graph, edge[0], edge[1], p=p, q=q)

    return alias_nodes, alias_edges

In [5]:
def alias_draw(J, q):
    """
    draw sample from a non-uniform discrete distribution using alias sampling.
    """
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

def generate_first_order_random_walk(graph, alias_nodes, walk_length=10, start_node=None):
    """
    simulate a random walk starting from start node and considering the first order information.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    cur = start_node
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0:
            # sample the next node based on alias_nodes
            cur = cur_nbrs[alias_draw(*alias_nodes[cur])]
            walk.append(cur)
        else:
            break

    return walk
    
def generate_second_order_random_walk(graph, alias_nodes, alias_edges, walk_length=10, start_node=None):
    """
    simulate a random walk starting from start node and considering the second order information.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    
    prev = None
    cur = start_node
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0:
            if prev is None:
                # sample the next node based on alias_nodes
                prev, cur = cur, cur_nbrs[alias_draw(*alias_nodes[cur])]
            else:
                # sample the next node based on alias_edges
                prev, cur = cur, cur_nbrs[alias_draw(*alias_edges[(prev, cur)])]
            walk.append(cur)
        else:
            break

    return walk

In [6]:
def build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10):
    """
    build a deepwalk model
    """
    print("building a DeepWalk model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_first_order_random_walk(
                graph, alias_nodes, walk_length=walk_length, start_node=node))
        
    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")
    
    # train a skip-gram model for these walks
    model = Word2Vec(walks, size=node_dim, window=3, min_count=0, sg=1, workers=os.cpu_count(), iter=16)
    print("trainig time: %.4f" % (time.time()-st))
    
    return model

def build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10):
    """
    build a node2vec model
    """
    print("building a node2vec model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_second_order_random_walk(
                graph, alias_nodes, alias_edges, walk_length=walk_length, start_node=node))
            
    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0    
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")
    
    # train a skip-gram model for these walks
    model = Word2Vec(walks, size=node_dim, window=3, min_count=0, sg=1, workers=os.cpu_count(), iter=16)
    print("trainig time: %.4f" % (time.time()-st))
    
    return model

In [7]:
def get_cosine_sim(model, u, v):
    """
    get the cosine similarity between two nodes
    """
    try:
        u = model.wv.vectors[model.wv.index2word.index(u)]
        v = model.wv.vectors[model.wv.index2word.index(v)]
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    except:
        return 0

def get_auc_score(model, true_edges, false_edges):
    """
    get the auc score
    """
    y_true = [1] * len(true_edges) + [0] * len(false_edges)
    
    y_score = list()
    for e in true_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    for e in false_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    print(y_true[:50])
    print(y_score[:50])
    return roc_auc_score(y_true, y_score)

In [8]:
def write_pred(file_name, edge, pred):
    #print(edge.type)
    print(len(edge))
    #print(edge[:10])
    head_list = list()
    end_list = list()
    for (head,end) in edge:
        head_list.append(head)
        end_list.append(end)
    #print(head_list)
    #print(end_list)
    #print(edge[:10][0])
    #print(edge[:10][1])
    df = pd.DataFrame(zip(head_list,end_list, pred))
    df.columns = ["src", "dst", "score"]
    df.to_csv(file_name, index=False)

In [9]:
train_file = "data/train.csv"
valid_file = "data/valid.csv"
edges = load_data(train_file)
graph = construct_graph_from_edges(edges)
num_false_edges = 16000
train_edges = load_data(train_file)
valid_edges = load_data(valid_file)
false_edges = generate_false_edges(train_edges+valid_edges, num_false_edges)
write_pred("valid_ans.csv", valid_edges+false_edges, [1]*len(valid_edges) + [0]*num_false_edges)

number of nodes: 8347
number of edges: 100000
35268


In [ ]:
alias_nodes, alias_edges = preprocess_transition_probs(graph, p=4, q=3)

In [ ]:
generate_first_order_random_walk(graph, alias_nodes=alias_nodes,
                                 start_node="N6ZTMIue-2b30CJv2tyPGg", walk_length=10)

In [ ]:
generate_second_order_random_walk(graph, alias_nodes=alias_nodes, alias_edges=alias_edges,
                                  start_node="N6ZTMIue-2b30CJv2tyPGg", walk_length=10)

In [ ]:
model = build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10)

In [ ]:
model = build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10)

In [ ]:
print("node embedding (\"N6ZTMIue-2b30CJv2tyPGg\"):",
      model.wv.vectors[model.wv.index2word.index("N6ZTMIue-2b30CJv2tyPGg")])
print("node embedding (\"N7E-CfqdME28dakWdEKNvw\"):",
      model.wv.vectors[model.wv.index2word.index("N7E-CfqdME28dakWdEKNvw")])
print("node embedding (\"MmlJSLDg-IFaeXb5wdJbgg\"):",
      model.wv.vectors[model.wv.index2word.index("MmlJSLDg-IFaeXb5wdJbgg")])
print("true edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"N7E-CfqdME28dakWdEKNvw\"):",
      get_cosine_sim(model, "N6ZTMIue-2b30CJv2tyPGg", "N7E-CfqdME28dakWdEKNvw"))
print("false edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"MmlJSLDg-IFaeXb5wdJbgg\"):",
      get_cosine_sim(model, "N6ZTMIue-2b30CJv2tyPGg", "MmlJSLDg-IFaeXb5wdJbgg"))

In [10]:
np.random.seed(0)

kfold = 5
node_dim = 8
num_walks = 16
walk_length = 8
p = 1
q = 1

deepwalk_auc_scores = list()
node2vec_auc_scores = list()
kf = KFold(n_splits=kfold, shuffle=True)
for k, (train_idx, valid_idx) in enumerate(kf.split(edges)):
    # split edges into training and validation
    train_edges = [edges[idx] for idx in train_idx]
    valid_edges = [edges[idx] for idx in valid_idx]
    # generate the same validation size of false edges
    false_edges = generate_false_edges(edges, num_false_edges=len(valid_edges))
    
    # construct the graph and preprocess transition probabilities
    graph = construct_graph_from_edges(train_edges)
    alias_nodes, alias_edges = preprocess_transition_probs(graph, p=p, q=q)
    
    # build models and get auc scores
    model = build_deepwalk(graph, alias_nodes,
                           node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
    deepwalk_auc_scores.append(get_auc_score(model, valid_edges, false_edges))
    
    model = build_node2vec(graph, alias_nodes, alias_edges,
                           node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
    node2vec_auc_scores.append(get_auc_score(model, valid_edges, false_edges))
    
deepwalk_auc_scores = np.array(deepwalk_auc_scores)
node2vec_auc_scores = np.array(node2vec_auc_scores)

number of nodes: 8053
number of edges: 80000
building a DeepWalk model...	number of walks: 128848	average walk length: 6.9806	trainig time: 40.0041
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.8986084, 0.9513591, 0.8442014, 0.9839856, 0.66950643, 0.84827226, 0.8129294, 0.8562799, 0.9385861, 0.97933453, 0.9843266, 0.8966223, 0.9700726, 0.7828295, 0.94852793, 0.94415724, 0.97568095, 0.9830596, 0.89123446, 0.9775854, 0.921565, 0.9359797, 0.96124804, 0.9495148, 0.9846981, 0.93618226, 0.6959616, 0.91733724, 0, 0.6410515, 0.984565, 0.97883976, 0.9810273, 0.9172357, 0.9752598, 0.9727724, 0.98339117, 0.8641984, 0.8361701, 0.9218181, 0.8762497, 0.84315664, 0.87925255, 0.9224618, 0.9025227, 0.89668703, 0.92849404, 0.8467735, 0.95890796, 0.9533327]
building a node2vec model...	number of walks: 128848	average walk length: 6.9806	trainig time: 47.5561
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:
print("DeepWalk: avg auc score: %.4f\tstd: %.4f" % (deepwalk_auc_scores.mean(), deepwalk_auc_scores.std()))

DeepWalk: avg auc score: 0.9204	std: 0.0011


In [12]:
print("node2vec: avg auc score: %.4f\tstd: %.4f" % (node2vec_auc_scores.mean(), node2vec_auc_scores.std()))

node2vec: avg auc score: 0.9204	std: 0.0011


In [13]:
model = build_deepwalk(graph, alias_nodes,
                       node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
print(get_auc_score(model, valid_edges, false_edges))

model = build_node2vec(graph, alias_nodes, alias_edges,
                       node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
print(get_auc_score(model, valid_edges, false_edges))

building a DeepWalk model...	number of walks: 129216	average walk length: 6.9649	trainig time: 44.6125
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.87696826, 0.7865533, 0.842802, 0.73463285, 0.89634895, 0.8814998, 0.99387527, 0.825043, 0.74016935, 0.9586896, 0.94517845, 0.99095, 0.9614918, 0.935879, 0.95395386, 0.78380877, 0.94496214, 0.8428003, 0.9577706, 0.9397345, 0.91314614, 0.91922575, 0.56058264, 0.8727027, 0.8549203, 0.7839532, 0.84180784, 0.9006921, 0.98220146, 0.98031414, 0.8736851, 0.94997317, 0.5763305, 0.9368904, 0.9778384, 0.9053665, 0.9385288, 0.93533885, 0.79925936, 0.9047257, 0.6653595, 0.29448116, 0.93972915, 0.8448847, 0.8735462, 0.9312013, 0.97243094, 0.957272, 0.88843405, 0.9732779]
0.92010264875
building a node2vec model...	number of walks: 129216	average walk length: 6.9649	trainig time: 44.5822
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
ans = pd.read_csv("data/valid_ans.csv", usecols=["src", "dst", "score"])
pred = pd.read_csv("data/valid_pred.csv", usecols=["src", "dst", "score"])
df = pd.merge(ans, pred, how="left", on=["src", "dst"])
df.fillna(0, inplace=True)
auc_score = roc_auc_score(df["score_x"], df["score_y"])
print("auc score:", auc_score)